# SETUP

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import IPython.display as disp
import time
import concurrent.futures

from acqpack import Manifold, Mfcs
from acqpack import Motor, AsiController, Autosampler, FractionCollector
from acqpack import utils as ut
from acqpack import gui
from acqpack import Log

l = Log()
l.track_classes([Manifold, Mfcs, Autosampler, FractionCollector]) # only call once
l.show_filter = '''~(cl=='Manifold')'''

### Manifold + Device Control
- **Device**: MITOMI (inlet bus only)
- **bank_0-7**: 32psi, filled with water
    - **chip control** as in `valvemap.txt`: Tygon
- **bank_8-15**: 32psi, __8__ filled with water, **9** filled with air

In [3]:
m = Manifold("192.168.1.2", 'config/run/valvemap.txt')
# gui.manifold_control(m)
m.valvemap.query('name!=" "')

@l.track_fn
def open_waste():
    for i in range(5):
        m.open('valve',i)
        time.sleep(.25)
    
@l.track_fn
def close_waste():
    for i in range(4,-1,-1):
        m.close('valve',i)
        time.sleep(.25)
        
m.open_waste = open_waste
m.close_waste = close_waste

### MFCS + Device Flow
- __mfcs[1]__ to __125ml Nalgene-PETG reservoir (water)__: Tygon
- __125ml Nalgene-PETG reservoir filled (water)__ to __device[in5]__: Tygon
- __mfcs[2]__ to __autosampler\[headspace-needle\]__ : Tygon
- __autosampler[sample-needle]__ to __device[in6]__: PEEK tubing, 65 um dia-in, 500 um dia-out, 35 cm len
- __device[in7]__ to __collector[dropper]__: PEEK tubing, 65 um dia-in, 500 um dia-out, 35 cm len
- __manifold[9]__ to __collector[sheath]__: Tygon
- __device[in4]__ to __50ml conical (empty)__: Tygon

In [6]:
p = Mfcs('config/system/mfcs.yaml', 'config/run/chanmap.txt')
p.chanmap

MFCS initialized. SN: 758
PID alpha: 2
Pressure units: psi


,channel,inlet
0,0,all
1,1,wash
2,2,sampler
3,3,
4,4,


### Autosampler

In [7]:
a = Autosampler(Motor('config/system/sampler-motor.yaml'),  AsiController('config/system/sampler-asi.yaml'))
a.add_frame('wash', 'config/system/sampler-deck.txt', 'config/run/sampler-4scint.txt')
a.add_frame('plate', 'config/system/sampler-deck.txt', 'config/run/sampler-96well.txt')
a.zh_travel = 37 # hardware safe travel height
gui.stage_control(a)

(0.000, 0.000, 0.000) hardware
(0.330, -28.698, 99.600) wash
(0.330, -28.698, 99.600) plate


### Fraction Collector

In [8]:
f = FractionCollector(AsiController('config/system/collector-asi.yaml'))
f.add_frame('plate', 'config/system/collector-deck.txt', 'config/run/collector-96well.txt')
gui.stage_control(f)

(0.000, 0.000) hardware
(0.000, 0.000) plate


### Micromanager
- run in `microscopy.ipynb`

In [9]:
start_checklist = '''
Fluid Levels -------------------
- Scint Vials
- Wash-In Vessel
- Waste Vial
- Plates

Manifold -----------------------
- Valves: 35psi, reservoir full
- Sheath: 35psi, reservoir empty
-   MFCS: 17psi (<=1300 mbar)

Scope --------------------------
- ROI
- HAL off
- LED 100% 590nm, FL open
- Acq Running

Press ENTER to start run
'''

end_checklist = '''
Fluid Levels -------------------
- Plates: remove

Manifold -----------------------
- Valves: 25psi

Scope --------------------------
- LED off
- HAL off
''' 

# SCRIPT

In [10]:
@l.track_fn
def simul(fargs):
    with concurrent.futures.ThreadPoolExecutor(max_workers=len(fargs)) as executor:
        for f,args in fargs.iteritems():
            executor.submit(f, *args)

simul({a.goto: ('wash', 'contents', 'waste'),
       f.goto: ('plate','series','waste')})

In [11]:
p_low_sampler = .1
p_low = 1.5
p_high = 15

@l.track_fn
def idle():
    m_idle()
    p_idle()
    simul({a.home:(),
           f.goto:('plate','series','waste')})
    sheath()
    
def m_idle():
    m.close('outlet', 'collector')
    m.close('inlet','sampler')
    m.close('inlet','wash')
    m.close_waste()
    m.open('name','sheath') # depressurize
    m.close('name','in')
    
def p_idle():
    p.set('inlet','sampler', p_low_sampler)
    p.set('inlet','wash', p_low)

# def from_wash():
#     pass

# def from_sampler():
#     pass

@l.track_fn  
def to_waste(sec):
    m.open_waste()
    time.sleep(sec)
    m.close_waste()
    
@l.track_fn  
def to_sampler(sec):
    m.open('outlet','sampler')
    time.sleep(sec)
    m.close('outlet','sampler')
    
@l.track_fn  
def to_collector(sec, use_sheath=True):
    m.open('outlet','collector')
    if use_sheath:
        sheath(sec)
    else:
        time.sleep(sec)
    m.close('outlet','collector') 

@l.track_fn    
def needle_wash():
    a.goto('wash', 'contents', 'etoh')
    for _ in range(4):
        a.Z.move_relative(-3.)
        a.Z.move_relative(3.)
    a.goto('wash', 'contents', 'water')
    for _ in range(4):
        a.Z.move_relative(-3.)
        a.Z.move_relative(3.)

@l.track_fn
def sheath(sec=0):
    sheath_period = 28
    sheath_length = .09
    
    if sec>0:
        t_end = time.time() + sec
        while time.time() < t_end-sheath_period-sheath_length-.1:
                time.sleep(sheath_period)
                m.close('name','sheath')
                time.sleep(sheath_length)
                m.open('name','sheath')
        time.sleep(t_end-time.time())
        
    m.close('name','sheath')
    time.sleep(sheath_length)
    m.open('name','sheath')

In [12]:
@l.track_fn
def prime(well_in, sec_waste=175, sec_collector=55):
    a.goto('plate', 'well', well_in['well'])
    p.set('inlet', 'sampler', p_high)
    m.open('inlet','sampler')
    
    to_waste(sec_waste)
    to_collector(sec_collector, use_sheath)

@l.track_fn
def collect(well_out, sec=60*3.88):
    f.goto('plate','well', well_out['well'])
    to_collector(sec, use_sheath)
    
    p.set('inlet','sampler', p_low_sampler)
    time.sleep(1) # give sampler pressure time to decrease
    m.close('inlet','sampler')
    
@l.track_fn
def flush(sec_waste=40, sec_sampler=60*3.75, sec_collector=60*3.75, sec_air_purge=4):
    m.open('inlet', 'wash')
    p.set('inlet','wash', p_high)

    simul({to_waste:(sec_waste,),
           a.goto:('wash', 'contents', 'waste'),
           f.goto:('plate','well', well_waste)})
    
    simul({to_sampler:(sec_sampler,),
           to_collector:(sec_collector, use_sheath)})
    
    if use_needle_wash:
        simul({needle_wash:(),
               time.sleep:(sec_air_purge,)})
    
    p.set('inlet','wash', p_low)
    m.close('inlet','wash')
    
# DEVICE CLASS
# think of flow graph / flow path through graph
# set node(inlet(s) + outlet(s)) parameters, connection matrix
# instead of m.open()/m.close(), hold valves open for lengths of time? optional m.open(sec=0)?

In [13]:
@l.track_fn
def setup(show_checklist=True, sec_air_purge=20):
    if show_checklist:
        raw_input(start_checklist)
        disp.clear_output()

    idle()
    flush(20,60,60,sec_air_purge)
    l.show()
    
@l.track_fn
def iteration(well_in, well_out):
    prime(well_in)
    l.show()
    collect(well_out)
    l.show()
    flush()
    l.show()
    
@l.track_fn
def cleanup(show_checklist=True):    
    flush(20,60,60)
    idle()
    
    if show_checklist:
        print end_checklist
        
    # save
    ts = time.strftime("%Y%m%d-%H%M", time.localtime())
    l.df.to_csv('data/{}_log.csv'.format(ts), index=False, header=True) # log
    
    l.show()
    l.write('DONE')

# Run --------------------------------------

In [14]:
q = '''well.str.contains('B|C|D|E')'''
# q = '''well in ['C01','C03']'''

map_in = (a.frames.plate.position_table
          .query(q)
          .sort_index(ascending=True).reset_index())
map_out = (f.frames.plate.position_table
           .query(q)
           .sort_index(ascending=True).reset_index())

disp.display(map_in)
disp.display(map_out)

,index,n,s,r,c,well,series,conc_fluorophore,conc_dna,vol,amt_fluorophore,amt_dna,needle_wash,sheath,x,y,z
0,12,12,23,1,0,B01,+n-s,0.000000e+00,0.000000e+00,0.00003,0.000000e+00,0.000000e+00,1.0,1.0,2.493,21.749,-0.85
1,13,13,22,1,1,B02,+n-s,0.000000e+00,0.000000e+00,0.00003,0.000000e+00,0.000000e+00,1.0,1.0,11.493,21.749,-0.85
2,14,14,21,1,2,B03,+n-s,2.500000e-05,1.000000e-12,0.00003,7.500000e-10,3.000000e-17,1.0,0.0,20.493,21.749,-0.85
3,15,15,20,1,3,B04,+n-s,0.000000e+00,0.000000e+00,0.00003,0.000000e+00,0.000000e+00,1.0,0.0,29.493,21.749,-0.85
4,16,16,19,1,4,B05,+n-s,0.000000e+00,0.000000e+00,0.00003,0.000000e+00,0.000000e+00,1.0,0.0,38.493,21.749,-0.85
5,17,17,18,1,5,B06,+n-s,2.500000e-05,1.000000e-12,0.00003,7.500000e-10,3.000000e-17,1.0,0.0,47.493,21.749,-0.85
6,18,18,17,1,6,B07,+n-s,0.000000e+00,0.000000e+00,0.00003,0.000000e+00,0.000000e+00,1.0,0.0,56.493,21.749,-0.85
7,19,19,16,1,7,B08,+n-s,0.000000e+00,0.000000e+00,0.00003,0.000000e+00,0.000000e+00,1.0,0.0,65.493,21.749,-0.85
8,20,20,15,1,8,B09,+n-s,2.500000e-05,1.000000e-12,0.00003,7.500000e-10,3.000000e-17,1.0,0.0,74.493,21.749,-0.85
9,21,21,14,1,9,B10,+n-s,0.000000e+00,0.000000e+00,0.00003,0.000000e+00,0.000000e+00,1.0,0.0,83.493,21.749,-0.85


,index,n,s,r,c,well,well_waste,series,conc_fluorophore,conc_dna,...,amt_fluorophore,amt_dna,needle_wash,sheath,x,y,z,intensity,plate_reader,qpcr
0,12,12,23,1,0,B01,A01,+n-s,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,1.0,1.0,2.493,21.749,-0.85,NaN,NaN,NaN
1,13,13,22,1,1,B02,A02,+n-s,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,1.0,1.0,11.493,21.749,-0.85,NaN,NaN,NaN
2,14,14,21,1,2,B03,A03,+n-s,2.500000e-05,1.000000e-12,...,2.500000e-17,1.000000e-24,1.0,0.0,20.493,21.749,-0.85,NaN,NaN,NaN
3,15,15,20,1,3,B04,A04,+n-s,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,1.0,0.0,29.493,21.749,-0.85,NaN,NaN,NaN
4,16,16,19,1,4,B05,A05,+n-s,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,1.0,0.0,38.493,21.749,-0.85,NaN,NaN,NaN
5,17,17,18,1,5,B06,A06,+n-s,2.500000e-05,1.000000e-12,...,2.500000e-17,1.000000e-24,1.0,0.0,47.493,21.749,-0.85,NaN,NaN,NaN
6,18,18,17,1,6,B07,A07,+n-s,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,1.0,0.0,56.493,21.749,-0.85,NaN,NaN,NaN
7,19,19,16,1,7,B08,A08,+n-s,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,1.0,0.0,65.493,21.749,-0.85,NaN,NaN,NaN
8,20,20,15,1,8,B09,A09,+n-s,2.500000e-05,1.000000e-12,...,2.500000e-17,1.000000e-24,1.0,0.0,74.493,21.749,-0.85,NaN,NaN,NaN
9,21,21,14,1,9,B10,A10,+n-s,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,1.0,0.0,83.493,21.749,-0.85,NaN,NaN,NaN


In [15]:
use_sheath, use_needle_wash, well_waste  = True, True, 'A01'
setup()
for (i, well_in), (j, well_out) in zip(map_in.iterrows(), map_out.iterrows()):
    use_sheath, use_needle_wash, well_waste = well_in.sheath, well_in.needle_wash, well_out.well_waste
    iteration(well_in, well_out)

use_sheath, use_needle_wash, well_waste  = True, True, 'A12'
cleanup()
l.write('DONE')

# Shutdown

In [ ]:
del p

# ---------------

In [ ]:
s = '2h I X\r'
# s = '2h S X=7.3 Y=7.3\r'

a.XY.serial.write(s)
time.sleep(.5)
resp = a.XY.serial.read(a.XY.serial.in_waiting)
print resp.replace('\r','\n\r')

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import IPython.display as disp
df = pd.read_csv('temp.csv').fillna('')
df['in'] = df['in'].apply(lambda x: tuple(y for y in x.replace('(','').replace(')','').split(', ')))

# vlines label
def plot(dd):
    plt.figure(figsize=(15,3))
    yt=[]
    for i,row in pd.DataFrame(dd.groupby('name')).iterrows():
        yt.append(row[0])
        for j,(x0,x1,s) in enumerate(zip(row[1].t_in - t0, row[1].t_out - t0, row[1]['fn'])):
            y0=i+j*0
            plt.vlines(x0, y0+.25,y0, color='green', lw=.4)
            plt.hlines(y0, x0,x1, color='b', lw=1)
            plt.vlines(x1, y0,y0-.25, color='red', lw=.4)
            plt.annotate(xy=(x0+.2,y0+.02),s=s)
    plt.yticks(np.arange(len(yt)), yt)
    plt.show()

In [ ]:
t0 = df.t_in.min()
yt = []
for _,c in pd.DataFrame(df.query('fn!="sheath"')
                        .query('fn!="pressurize"')
                        .query('fn!="depressurize"')
                        .query('fn!="read_valve"')
                        .groupby('class')).iterrows():
    
    dd=c[1]
    
    if c[0]=='':
        dd['name'] = dd['fn']
        plot(dd)
    
#     if c[0]=='Autosampler':
#         dd['name'] = 'move'
#         plot(dd)

#     if c[0]=='FractionCollector':
#         dd['name'] = 'move'
#         plot(dd)
        
#     if c[0]=='Manifold':
#         dd = dd.dropna()
#         dd=dd[dd['in'].apply(lambda x: 'r'!=x[-1][1])]
#         dd['name'] = dd['in'].apply(lambda x: x[-1].replace('\'',''))
#         dd = dd.query('name!="sheath"')
#         plot(dd)

#     if c[0]=='Mfcs':
#         dd['name'] = dd['in'].apply(lambda x: x[1].replace('\'',''))
#         plot(dd)

In [ ]:
def plot(dd):
    plt.figure(figsize=(15,3))
    yt=[]
    for i,row in pd.DataFrame(dd.groupby(['name','class'])).iterrows():
        yt.append(row[0])
        for j,(x0,x1,s) in enumerate(zip(row[1].t_in - t0, row[1].t_out - t0, row[1]['fn'])):
            y0=i+j*0
            plt.vlines(x0, y0+.25,y0, color='green', lw=.4)
            plt.hlines(y0, x0,x1, color='b', lw=1)
            plt.vlines(x1, y0,y0-.25, color='red', lw=.4)
            plt.annotate(xy=(x0+.2,y0+.02),s=s)
    plt.yticks(np.arange(len(yt)), yt)
    plt.show()

t0 = l.df.t_in.min()
yt = []
dd = l.df 
dd['name'] = dd['th']
plot(dd)

In [ ]:
%matplotlib tk

In [ ]:
# Timing test
m.close('inlet','wash')
p.set('inlet','wash', 2)

p.set('inlet','sampler',.2)
a.goto('plate','well', 'B03')
p.set('inlet','sampler',15.5)
m.open('inlet','sampler')

# prime sampler
m.open_waste()
time.sleep(155)
# t0 = time.time()
# raw_input('note: fluorescence plateaus')
# print 'prime sampler', time.time()-t0
m.close_waste()

# prime collector
m.open('outlet','collector')
t0 = time.time()
raw_input('note: color at outlet')
print 'prime collector', time.time()-t0
m.close('outlet','collector')

m.close('inlet', 'sampler')
p.set('inlet','sampler',.2)
time.sleep(.2)
a.goto('wash','contents', 'waste')

p.set('inlet','wash', 16)
m.open('inlet','wash')

m.open_waste()
time.sleep(10)
m.close_waste()

# flush sampler + collector
m.open('outlet','sampler')
m.open('outlet','collector')
t0 = time.time()
raw_input('note: clear at needle')
print 'flush sampler + collector (sampler)', time.time()-t0
m.close('outlet','sampler')
m.close('outlet','collector')

In [ ]:
m.open('outlet','collector')
sheath(3.88*60)
m.close('outlet','collector')